In [ ]:
# Lib Imports
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression, LinearRegression, LassoCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import category_encoders as ce

In [ ]:
# Setting Pandas column display option
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [ ]:
%%time
idee = pd.read_csv('train_identity.csv')
transaction = pd.read_csv('train_transaction.csv')

In [ ]:
%%time
def pipeline(idee, transaction):
    merge = transaction.merge(idee, how='outer', on='TransactionID')
    objects = merge.select_dtypes('object')
    objects = objects.join(merge[['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2']])
    objects['isFraud'] = merge['isFraud']
    objects.fillna("Unknown", inplace=True)
    objects = objects.astype('category')
    print(objects.info())
    return objects
objects = pipeline(idee,transaction)

In [ ]:
objects.drop(columns=['M4','id_15','id_16','id_28','id_29','id_35','id_36','id_38','DeviceType','DeviceInfo'], inplace=True)
objects.shape

In [ ]:
#value_counts_list = []
#obj_columns_list = objects.columns.to_list()
#for column in obj_columns_list:
    #df = objects[column].value_counts().reset_index()
    #value_counts_list.append(df)

#objects_value_counts_df = pd.concat(value_counts_list, axis=1)
#objects_value_counts_df.shape

In [ ]:
%%time
X  = objects.drop(columns='isFraud')
y = objects['isFraud'].astype('int')

under = RandomUnderSampler(sampling_strategy=0.8) 
over = RandomOverSampler(sampling_strategy=0.5)
# sm = SMOTE()
# ad = ADASYN()
# lasso = LassoCV(tol=0.01, n_jobs=-1)

#X_rus, y_rus = under.fit_resample(X, y)
X_ros, y_ros = over.fit_resample(X, y)
# X_sm, y_sm = sm.fit_resample(X, y)
# X_ad, y_ad = ad.fit_resample(X, y)
X_cs, y_cs = under.fit_resample(X_ros, y_ros)
del X_ros, y_ros

def split(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
    return X_train, X_test, y_train, y_test

logit = LogisticRegression(
        penalty='l2', C=1e42, max_iter=500, verbose=1, solver='liblinear', n_jobs=-1)

In [ ]:
X_train, X_test, y_train, y_test = split(X_cs, y_cs)

columns = X_train.columns.to_list()

def get_score(model, X, y, X_test, y_test):
    model.fit(X, y)
    y_pred = model.predict_proba(X_test)[:,1]
    score = roc_auc_score(y_test, y_pred)
    return score

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
y_train.head()

In [ ]:
y_test.head()

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
%%time
# n_components_list = np.arange(100, 500, 100).tolist()
n_components_list = [1000]
n_components_list_str = [str(i) for i in n_components_list]

fh_logit_scores = []

for n_components in n_components_list:
    hashing_enc = ce.HashingEncoder(cols=columns, n_components=n_components, max_process=4).fit(X_train, y_train)
    
    X_train_hashing = hashing_enc.transform(X_train.reset_index(drop=True))
    print(X_train_hashing.shape)
    X_test_hashing = hashing_enc.transform(X_test.reset_index(drop=True))
    print(X_test_hashing.shape)

    fe_logit_score = get_score(logit, X_train_hashing, y_train, X_test_hashing, y_test)
    fh_logit_scores.append(fe_logit_score)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(n_components_list_str, fh_logit_scores, linewidth=3)
plt.title('n_compontents vs roc_auc for feature hashing with logistic regression')
plt.xlabel('n_components')
plt.ylabel('score')
plt.show()

In [ ]:
fh_logit_scores

In [ ]:
growth_rate = np.exp(np.diff(np.log(fh_logit_scores))) - 1

In [ ]:
growth_rate

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(n_components_list_str, growth_rate, linewidth=3)
plt.title('n_compontents vs growth_rate for feature hashing with logistic regression')
plt.xlabel('n_components')
plt.ylabel('GRate')
plt.show()

## Test dataset benchmarking

In [ ]:
%%time
hashing_enc = ce.HashingEncoder(cols=columns, n_components=1600, max_process=4).fit(X_train, y_train)
X_train_hashing = hashing_enc.transform(X_train.reset_index(drop=True))
X_test_hashing = hashing_enc.transform(X_test.reset_index(drop=True))
print(X_train_hashing.shape, y_train.shape)

In [ ]:
%%time
idee_test = pd.read_csv('test_identity.csv')
transaction_test = pd.read_csv('test_transaction.csv') 
merge_test = transaction_test.merge(idee_test, how='outer', on='TransactionID')
objects_test = merge_test.select_dtypes('object')
objects_test.fillna("Unknown", inplace=True)
objects_test = objects_test.astype('category')
objects_test.columns = X_train.columns
print(objects_test.info())
del idee_test, transaction_test

In [ ]:
X2_hashed = hashing_enc.transform(objects_test.reset_index(drop=True), override_return_df=True)

In [ ]:
%%time
X2_hashed.head()

In [ ]:
%%time
logit.fit(X_train_hashing, y_train)
y_pred_train = logit.predict(X_test_hashing)
from sklearn.metrics import classification_report
print(classification_report(y_pred_train, y_test))

In [ ]:
%%time
y_pred = logit.predict(X2_hashed)

In [ ]:
y_pred